In [ ]:
import scipy as sci
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import odeint
import time
from pynverse import inversefunc
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import animation

def Gravity(w, t, m):
    num = len(m)
    r = np.zeros((num,3))
    v = np.zeros((num,3))
    dvdt = np.zeros((num,3))
    x = w[0::6]
    y = w[1::6]
    z = w[2::6]
    r = np.array([x,y,z]).transpose()
    vx =w[3::6]
    vy =w[4::6]
    vz =w[5::6]
    v = np.array([vx,vy,vz])
    for i in range(num):
        for j in range(num):
            if i != j:
                dvdt[i] += m[j]*(r[i]-r[j])/sci.linalg.norm(r[i]-r[j])**3
    dvdt = -K1*dvdt.transpose()
    dxdt = K2*v
    derivs = sci.concatenate((dxdt,dvdt)).transpose().flatten()
    return derivs

def ModMid(func, W, t, H, n):
    h = H/n
    w = np.zeros((n+1, len(W)))
    w[0] = W 
    w[1] = w[0] + h*func(w[0], h)
    for i in range(2, n+1):
        w[i] = w[i-2] + 2*h*func(w[i-1], h)
    WH = 1/2*(w[n]+w[n-1]+h*func(w[n], h))
    return WH

def rk4(func, w, t, h):
    fa = func(w, t)
    Wb = w + h/2*fa
    fb = func(Wb, t+h/2)
    Wc = w +h/2*fb
    fc = func(Wc, t+h/2)
    Wd = w + h*fc
    fd = func(Wd, t+h)
    W = w + 1/6*h*fa+1/3*h*fc+1/3*h*fc +1/6*h*fd
    return W
def Euler(func, w, t, h):
    W = w + h*func(w, t)
    return W

def RE(func, diff, w, subres, H, oke=False):
    sol = np.zeros(len(w))
    subsol =  np.zeros((subres,len(w)))
    h = np.zeros(subres)
    for i in range(0,subres):
        n = (i+1)*2
        subtime = np.linspace(0,H,n+1)
        h[i] = subtime[1]-subtime[0]
        W = np.zeros((len(subtime),len(w)))
        W[0] = w
        for k in range(1,len(subtime)):
            W[k] = diff(func, W[k-1], subtime, h[i])
        subsol[i] = W[-1]
    cek = np.linspace(0,h[0],100)
    for i in range(len(w)):
        z = np.polyfit(h,subsol[:,i],subres-1)
        fit = np.poly1d(z)(cek)
        sol[i]= np.poly1d(z)(0)
        if oke and i == 0:
            plt.plot(h/h[0],subsol[:,0],'o')
            plt.plot(cek/h[0],fit)
            plt.xlim(0,1)
    return sol

def kec(r,v,theta):
    if np.isscalar(theta):
        if theta>=0 and theta<np.pi:
            ALPHA = np.arcsin(h/(r*v))
        else:
            ALPHA = np.pi - np.arcsin(h/(r*v))
        Vx = v*np.cos(THETA+ALPHA)
        Vy = v*np.sin(THETA+ALPHA)
        Vz = 0
        V = np.array([Vx,Vy,Vz]).transpose()
        return V
    else:
        if theta.any()>=0 and theta.any()<np.pi:
            ALPHA = np.arcsin(h/(r*v))
        else:
            ALPHA = np.pi - np.arcsin(h/(r*v))
        Vx = v*np.cos(theta+ALPHA)
        Vy = v*np.sin(theta+ALPHA)
        Vz = Vy*0
        V = np.array([Vx,Vy,Vz]).transpose()
        return V

def funct(T, e, E):
    t = T/(2*np.pi)*(E-e*np.sin(E))
    return t
def funcE(T,e, t):
    n = (t//T)
    t = t - (t//T)*T
    inverse = inversefunc(lambda E:funct(T,e,E), domain=[0, 2*np.pi])
    return inverse(t)+n*np.pi*2
def theta(e, E):
    Ec = np.arccos(e)
    THETA = np.array([])
    n = (E//(2*np.pi))
    for En in E:
        Et = En - (En//(np.pi*2))*2*np.pi
        if Et > Ec and Et < (2*np.pi-Ec):
            TH = np.arctan(np.sqrt(1-e**2)*np.sin(Et)/(np.cos(Et)-e)) + np.pi
            THETA = np.append(THETA, TH)
        else:
            TH = np.arctan(np.sqrt(1-e**2)*np.sin(Et)/(np.cos(Et)-e))
            if TH < 0:
                TH += 2*np.pi
            THETA = np.append(THETA, TH)
    return THETA+2*np.pi*n
def asinE(TH, e):
    Ec = np.arccos(e)
    n = (TH+np.pi-Ec)//(2*np.pi)
    TH = TH - 2*np.pi*n
    sinE = np.sqrt(1-e**2)*np.sin(TH)/(1+e*np.cos(TH))
    if np.isscalar(TH):
        if sinE<-1:
            return -np.pi/2
        elif sinE>1:
            return np.pi/2
        else:
            if TH >=(-np.pi+Ec) and TH<=(np.pi-Ec):
                return np.arcsin(sinE) + 2*np.pi*n
            else:
                return np.pi-np.arcsin(sinE)
    else:
        E = np.zeros(len(TH))
        for i in range(len(TH)):
            if sinE[i]<-1:
                E[i] = -np.pi/2
            elif sinE[i]>1:
                E[i] = np.pi/2
            else:
                if TH[i]>=(-np.pi+Ec) and TH[i]<=(np.pi-Ec):
                    E[i] = np.arcsin(sinE[i]) + 2*np.pi*n[i]
                else:
                    E[i] = np.pi-np.arcsin(sinE[i]) + 2*np.pi*n[i]
        return E    
def th(t, e, T):
    E = funcE(T, e, t)
    THETA = theta(e, E)
    return THETA

def Proj(A, R, V):
    x = R/sci.linalg.norm(R)
    z = np.cross(x, V)/sci.linalg.norm(np.cross(x, V))
    y = np.cross(z, x)
    
    Ax = np.dot(A, x)
    Ay = np.dot(A, y)
    Az = np.dot(A, z)
    
    Ap = np.array([Ax,Ay,Az])
    return Ap
def THE(r,v,a,e):
    R = sci.linalg.norm(r)
    cos = a*(1-e**2)/e/R-1/e
    if np.dot(v,r)>0:
        return np.arccos(cos)
    else:
        return -np.arccos(cos)
    
def Angle(A,B,z,yes=False):
    AB = sci.linalg.norm(A)*sci.linalg.norm(B)
    y = np.cross(z,A)
    sign = np.dot(y,B)
    cos = np.dot(A,B)/AB
    if sign<=0:
        return np.arccos(cos)
    else:
        if yes:
            return -np.arccos(cos)
        else:
            return 2*np.pi-np.arccos(cos)

In [ ]:
def Angle(A,B,z,yes=False):
    AB = sci.linalg.norm(A)*sci.linalg.norm(B)
    y = np.cross(z,A)
    sign = np.dot(y,B)
    cos = np.dot(A,B)/AB
    if sign<=0:
        return np.arccos(cos)
    else:
        if yes:
            return -np.arccos(cos)
        else:
            return 2*np.pi-np.arccos(cos)
        
sudut = np.arange(1,360*2,1)*np.pi/180
xu = np.array([1,0,0])
yu = np.array([0,1,0])
zu = np.array([0,0,1])
BS = np.zeros(len(sudut))
panah = np.zeros((len(sudut),3))
for i in range(len(BS)):
    panah[i] = xu*np.cos(sudut[i])+yu*np.sin(sudut[i])
    BS[i] = Angle(panah[i],panah[0],zu)
    
plt.figure()
plt.plot(sudut,BS)
plt.show()

In [ ]:
A = np.array([1,1,1])
B = np.array([1,-1,1/2])
cos = np.dot(A,B)/(sci.linalg.norm(A)*sci.linalg.norm(B))
R = np.array([0,1,0])
V = np.array([0,1,1])
Ak = Proj(A, R, V)
Bk = Proj(B, R, V)
cos2 = np.dot(Ak,Bk)/(sci.linalg.norm(Ak)*sci.linalg.norm(Bk))
print(cos)
print(cos2)

In [ ]:
G=6.67430e-11 #N-m2/kg2
msun=1.9885e+30 #kg #mass of the sun
au =149597870700 #m #satu au
vc = np.sqrt(G*msun/au) #Kecepatan orbit bumi m/s
td = 2*np.pi*au/vc #Periode Orbit Bumi
#Total besaran satuan
K1=G*td*msun/(au**2*vc)
K2=vc*td/au

a = float(input('a (au) = '))
e = float(input('e = '))
THETA = float(input('THETA (deg) = '))*np.pi/180
m1 = float(input('m1 (msun) = '))
m2 = float(input('m2 (msun) = '))
M = m1+m2
m = m1*m2/M
R0 = a*(1-e**2)/(1+e*np.cos(THETA))
V0 = np.sqrt((2/R0-1/a)*M)
h = np.sqrt(M*a*(1-e**2))
T = np.sqrt(a**3/M)
E0 = asinE(THETA, e)
t0 = (E0 - e*np.sin(E0))*T/2/np.pi
print('Periode = %f tahun'%T)

r0 = np.array([R0*np.cos(THETA),R0*np.sin(THETA),0])
v0 = kec(R0,V0,THETA)

r10 = -r0*m2/M
r20 = r0*m1/M
v10 = -v0*m2/M
v20 = v0*m1/M

In [ ]:
import time
N = 3 #int(input('Jumlah Objek = '))
rentang = float(input('Masukkan rentang waktu (thn) = '))
resolusi = float(input('Masukkan resolusi (thn) = '))
mulai = time.time()
massa = 1/1
m = np.random.normal(loc=massa, scale=massa/5, size = N)
m[0] = 1
m[1] = 102.413e24/msun 
m[2] = 1.307e22/msun
#Posisi Koordinat Bola
x = np.zeros(N)
x[0] = -3.731720774696272E-03
x[1] = 2.923383235384231E+01
x[2] = 1.294914582958932E+01
y = np.zeros(N)
y[0] = 7.455523637382408E-03
y[1] = -6.384436376452419
y[2] = -3.136697162106187E+01
z = np.zeros(N)
z[0] = 2.118497459793055E-05
z[1] = -5.422472072207720E-01
z[2] = -3.892180087778638E-01
X = np.array([x,y,z])
#Kecepatan
vx = np.zeros(N)
vx[0] = -8.372316507588935E-06*(au/24/3600/vc)
vx[1] = 6.493781348157333E-04*(au/24/3600/vc)
vx[2] = 2.963301943091312E-03*(au/24/3600/vc)
vy = np.zeros(N)
vy[0] = -1.907822235499329E-06*(au/24/3600/vc)
vy[1] = 3.085786101820753E-03*(au/24/3600/vc)
vy[2] = 5.195369669072389E-04*(au/24/3600/vc)
vz = np.zeros(N)
vz[0] = 2.309120798929570E-07*(au/24/3600/vc)
vz[1] = -7.858388868092842E-05*(au/24/3600/vc)
vz[2] = -9.165222315794960E-04*(au/24/3600/vc)
V = np.array([vx,vy,vz])

# R = np.array([0,1,0])
# v = np.array([0,1,1])

XT = X.transpose()
VT = V.transpose()
Xp = np.zeros_like(XT)
Vp = np.zeros_like(VT)
for i in range(N):
    Xp[i] = Proj(XT[i], XT[1]-XT[0], VT[1]-XT[0])
    Vp[i] = Proj(VT[i], XT[1]-XT[0], VT[1]-XT[0])
#     Xp[i] = Proj(XT[i], R, v)
#     Vp[i] = Proj(VT[i], R, v)
Xb = Xp.transpose()
Vb = Vp.transpose()

XCOM = np.sum(m*Xb[0,:])/np.sum(m)
YCOM = np.sum(m*Xb[1,:])/np.sum(m)
ZCOM = np.sum(m*Xb[2,:])/np.sum(m)
VXCOM = np.sum(m*Vb[0,:])/np.sum(m)
VYCOM = np.sum(m*Vb[1,:])/np.sum(m)
VZCOM = np.sum(m*Vb[2,:])/np.sum(m)
Xb[0,:] -= XCOM
Xb[1,:] -= YCOM
Xb[2,:] -= ZCOM
Vb[0,:] -= VXCOM
Vb[1,:] -= VYCOM
Vb[0,:] -= VZCOM

# Package initial parameters
W0=np.concatenate((Xb,Vb)).transpose().flatten()

t = np.arange(0,rentang,resolusi)
selesai = time.time()
print('%f s'%(selesai - mulai))

In [ ]:
sci.linalg.norm(Vb.transpose()[1])
# sci.linalg.norm(np.dot(Xb.transpose()[1],Vb.transpose()[1]))
# X

In [ ]:
sci.linalg.norm(V.transpose()[1])
# sci.linalg.norm(np.dot(X.transpose()[1],V.transpose()[1]))
# V

In [ ]:
np.sum(m*X[0,:])/np.sum(m)

In [ ]:
Xs = x[0]
Ys = y[0]
Zs = z[0]
Vxs = vx[0]
Vys = vy[0]
Vzs = vz[0]
rs = np.array([Xs,Ys,Zs])
vs = np.array([Vxs,Vys,Vzs])

Xn = x[1]
Yn = y[1]
Zn = z[1]
Vxn = vx[1]
Vyn = vy[1]
Vzn = vz[1]

rn = np.array([Xn,Yn,Zn])
vn = np.array([Vxn,Vyn,Vzn])

Rcek =sci.linalg.norm(rn-rs)
Vcek =sci.linalg.norm(vn-vs)
HN = np.cross(rn-rs,vn-vs)
H = sci.linalg.norm(HN)
sin = H/(Rcek*Vcek)

a = 1/(2/Rcek-Vcek**2)
e = np.sqrt(1-H**2/a)
print(a)
print(e)

In [ ]:
t = np.linspace(0,T*10,100000)
r = np.zeros((len(t),3))
v = np.zeros((len(t),3))

TH = th(t+t0, e, T)
R = a*(1-e**2)/(1+e*np.cos(TH))
V = np.sqrt((2/R-1/a)*M)
v = kec(R, V, TH)

r[:,0] = R[:]*np.cos(TH[:])
r[:,1] = R[:]*np.sin(TH[:])
plt.figure(figsize=(5,5))
plt.plot(r[:,0],r[:,1])
plt.axis('equal')
plt.plot(r[-1,0],r[-1,1], 'o')
plt.grid()
plt.show()
plt.figure(figsize=(5,5))
plt.plot(v[:,0],v[:,1])
plt.axis('equal')
plt.plot(v[-1,0],v[-1,1], 'o')
plt.grid()
plt.show()

In [ ]:
#ODEINT
mulai = time.time()
W = np.zeros((len(t),len(W0)))
W = odeint(lambda w, t: Gravity(w, t, m), W0, t)
selesai = time.time()
print('%f s'%(selesai-mulai))
EKo = np.zeros(len(t))
EPo = np.zeros(len(t))
r_solo = np.zeros((len(t),N, 3))
v_solo = np.zeros((len(t),N, 3))
rcom = np.zeros((len(t), 3))
vcom = np.zeros((len(t), 3))
for i in range(len(t)):
    for j in range(N):
        r_solo[i,j] = np.array([W[i,6*j],W[i,6*j+1],W[i,6*j+2]])
        v_solo[i,j] = np.array([W[i,6*j+3],W[i,6*j+4],W[i,6*j+5]])
    rcom[i,0] = np.sum(m*r_solo[i,:,0])/np.sum(m)
    rcom[i,1] = np.sum(m*r_solo[i,:,1])/np.sum(m)
    rcom[i,2] = np.sum(m*r_solo[i,:,2])/np.sum(m)
    vcom[i,0] = np.sum(m*v_solo[i,:,0])/np.sum(m)
    vcom[i,1] = np.sum(m*v_solo[i,:,1])/np.sum(m)
    vcom[i,2] = np.sum(m*v_solo[i,:,2])/np.sum(m)
    for j in range(N):
        r_solo[i,j] -= rcom[i]
        v_solo[i,j] -= vcom[i]
    for j in range(N):
        for k in range(j,N):
            if j != k:
                EPo[i] += -m[j]*m[k]/sci.linalg.norm(r_solo[i,j]-r_solo[i,k])
        EKo[i] += 1/2*m[j]*(v_solo[i,j,0]**2+v_solo[i,j,1]**2+v_solo[i,j,2]**2)
E = EKo+EPo
error = np.zeros(len(E)-1)
logErr = np.zeros(len(E)-1)
for i in range(1,len(E)):
    error[i-1] = abs((E[i]-E[0])/E[0])
    if error[i-1] == 0:
        logErr[i-1] = -16
    else:
        logErr[i-1] = np.log10(error[i-1])
plt.figure(figsize=(6,6))
plt.plot(t[1:], logErr)
plt.xscale('log')
plt.xlabel('Waktu (thn)')
plt.xlim(t[1],t[-1])
plt.ylabel('Log Error')
plt.title('ODEINT')
plt.grid(axis='both')
plt.show()

In [ ]:
#Euler
mulai = time.time()
W = np.zeros((len(t),len(W0)))
W[0] = W0
h = t[1]-t[0]
for i in range(1,len(t)):
    W[i] = Euler(lambda w, t: Gravity(w, t, m), W[i-1], t[i-1], h)
selesai = time.time()
print('%f s'%(selesai-mulai))
EKe = np.zeros(len(t))
EPe = np.zeros(len(t))
r_sole = np.zeros((len(t),N, 3))
v_sole = np.zeros((len(t),N, 3))
rcom = np.zeros((len(t), 3))
vcom = np.zeros((len(t), 3))
for i in range(len(t)):
    for j in range(N):
        r_sole[i,j] = np.array([W[i,6*j],W[i,6*j+1],W[i,6*j+2]])
        v_sole[i,j] = np.array([W[i,6*j+3],W[i,6*j+4],W[i,6*j+5]])
    rcom[i,0] = np.sum(m*r_sole[i,:,0])/np.sum(m)
    rcom[i,1] = np.sum(m*r_sole[i,:,1])/np.sum(m)
    rcom[i,2] = np.sum(m*r_sole[i,:,2])/np.sum(m)
    vcom[i,0] = np.sum(m*v_sole[i,:,0])/np.sum(m)
    vcom[i,1] = np.sum(m*v_sole[i,:,1])/np.sum(m)
    vcom[i,2] = np.sum(m*v_sole[i,:,2])/np.sum(m)
    for j in range(N):
        r_sole[i,j] -= rcom[i]
        v_sole[i,j] -= vcom[i]
    for j in range(N):
        for k in range(j,N):
            if j != k:
                EPe[i] += -m[j]*m[k]/sci.linalg.norm(r_sole[i,j]-r_sole[i,k])
        EKe[i] += 1/2*m[j]*(v_sole[i,j,0]**2+v_sole[i,j,1]**2+v_sole[i,j,2]**2)
E = EKe+EPe
error = np.zeros(len(E)-1)
logErr = np.zeros(len(E)-1)
for i in range(1,len(E)):
    error[i-1] = abs((E[i]-E[0])/E[0])
    if error[i-1] == 0:
        logErr[i-1] = -16
    else:
        logErr[i-1] = np.log10(error[i-1])
plt.figure(figsize=(6,6))
plt.plot(t[1:], logErr)
plt.xscale('log')
plt.xlabel('Waktu (thn)')
plt.xlim(t[1],t[-1])
plt.ylabel('Log Error')
plt.title('Euler')
plt.grid(axis='both')
plt.show()

In [ ]:
#Bulisch-Stoer
subres = int(input('N = '))
mulai = time.time()
# W0 = np.array([r10,v10,r20,v20]).flatten()
W = np.zeros((len(t),len(W0)))
W[0] = W0
h = t[1]-t[0]
W[1] = RE(lambda w, t: Gravity(w, t, m), Euler, W[0], subres, h, oke=True)
for i in range(1,len(t)):
    W[i] = RE(lambda w, t: Gravity(w, t, m), Euler, W[i-1], subres, h, oke=False)
selesai = time.time()
print('%f s'%(selesai-mulai))
#########RUTINITAS
EKeb = np.zeros(len(t))
EPeb = np.zeros(len(t))
r_soleb = np.zeros((len(t),N, 3))
v_soleb = np.zeros((len(t),N, 3))
rcom = np.zeros((len(t), 3))
vcom = np.zeros((len(t), 3))
for i in range(len(t)):
    for j in range(N):
        r_soleb[i,j] = np.array([W[i,6*j],W[i,6*j+1],W[i,6*j+2]])
        v_soleb[i,j] = np.array([W[i,6*j+3],W[i,6*j+4],W[i,6*j+5]])
    rcom[i,0] = np.sum(m*r_soleb[i,:,0])/np.sum(m)
    rcom[i,1] = np.sum(m*r_soleb[i,:,1])/np.sum(m)
    rcom[i,2] = np.sum(m*r_soleb[i,:,2])/np.sum(m)
    vcom[i,0] = np.sum(m*v_soleb[i,:,0])/np.sum(m)
    vcom[i,1] = np.sum(m*v_soleb[i,:,1])/np.sum(m)
    vcom[i,2] = np.sum(m*v_soleb[i,:,2])/np.sum(m)
    for j in range(N):
        r_soleb[i,j] -= rcom[i]
        v_soleb[i,j] -= vcom[i]
    for j in range(N):
        for k in range(j,N):
            if j != k:
                EPeb[i] += -m[j]*m[k]/sci.linalg.norm(r_soleb[i,j]-r_soleb[i,k])
        EKeb[i] += 1/2*m[j]*(v_soleb[i,j,0]**2+v_soleb[i,j,1]**2+v_soleb[i,j,2]**2)
E = EKeb+EPeb
error = np.zeros(len(E)-1)
logErr = np.zeros(len(E)-1)
for i in range(1,len(E)):
    error[i-1] = abs((E[i]-E[0])/E[0])
    if error[i-1] == 0:
        logErr[i-1] = -16
    else:
        logErr[i-1] = np.log10(error[i-1])
plt.figure(figsize=(6,6))
plt.plot(t[1:], logErr)
plt.xscale('log')
plt.xlabel('Waktu (thn)')
plt.xlim(t[1],t[-1])
plt.ylabel('Log Error')
plt.title('Euler N = %d'%subres)
plt.grid(axis='both')
plt.show()

In [ ]:
#RungeKutta4
mulai = time.time()
W = np.zeros((len(t),len(W0)))
W[0] = W0
h = t[1]-t[0]
for i in range(1,len(t)):
    W[i] = rk4(lambda w, t: Gravity(w, t, m), W[i-1], t[i-1], h)
selesai = time.time()
print('%f s'%(selesai-mulai))
EKr = np.zeros(len(t))
EPr = np.zeros(len(t))
r_solr = np.zeros((len(t),N, 3))
v_solr = np.zeros((len(t),N, 3))
rcom = np.zeros((len(t), 3))
vcom = np.zeros((len(t), 3))
for i in range(len(t)):
    for j in range(N):
        r_solr[i,j] = np.array([W[i,6*j],W[i,6*j+1],W[i,6*j+2]])
        v_solr[i,j] = np.array([W[i,6*j+3],W[i,6*j+4],W[i,6*j+5]])
    rcom[i,0] = np.sum(m*r_solr[i,:,0])/np.sum(m)
    rcom[i,1] = np.sum(m*r_solr[i,:,1])/np.sum(m)
    rcom[i,2] = np.sum(m*r_solr[i,:,2])/np.sum(m)
    vcom[i,0] = np.sum(m*v_solr[i,:,0])/np.sum(m)
    vcom[i,1] = np.sum(m*v_solr[i,:,1])/np.sum(m)
    vcom[i,2] = np.sum(m*v_solr[i,:,2])/np.sum(m)
    for j in range(N):
        r_solr[i,j] -= rcom[i]
        v_solr[i,j] -= vcom[i]
    for j in range(N):
        for k in range(j+1,N):
            EPr[i] += -m[j]*m[k]/sci.linalg.norm(r_solr[i,j]-r_solr[i,k])
        EKr[i] += 1/2*m[j]*(v_solr[i,j,0]**2+v_solr[i,j,1]**2+v_solr[i,j,2]**2)
E = EKr+EPr
error = np.zeros(len(E)-1)
logErr = np.zeros(len(E)-1)
for i in range(1,len(E)):
    error[i-1] = abs((E[i]-E[0])/E[0])
    if error[i-1] == 0:
        logErr[i-1] = -16
    else:
        logErr[i-1] = np.log10(error[i-1])
plt.figure(figsize=(6,6))
plt.plot(t[1:], logErr)
plt.xscale('log')
plt.xlabel('Waktu (thn)')
plt.xlim(t[1],t[-1])
plt.ylabel('Log Error')
plt.title('Runge-Kutta orde 4')
plt.grid(axis='both')
plt.show()

In [ ]:
#Bulisch-Stoer
subres = int(input('N = '))
mulai = time.time()
W = np.zeros((len(t),len(W0)))
W[0] = W0
h = t[1]-t[0]
W[1] = RE(lambda w, t: Gravity(w, t, m), rk4, W[0], subres, h, oke=True)
for i in range(1,len(t)):
    W[i] = RE(lambda w, t: Gravity(w, t, m), rk4, W[i-1], subres, h, oke=False)
selesai = time.time()
print('%f s'%(selesai-mulai))
#########RUTINITAS
EKrb = np.zeros(len(t))
EPrb = np.zeros(len(t))
r_solrb = np.zeros((len(t),N, 3))
v_solrb = np.zeros((len(t),N, 3))
rcom = np.zeros((len(t), 3))
vcom = np.zeros((len(t), 3))
for i in range(len(t)):
    for j in range(N):
        r_solrb[i,j] = np.array([W[i,6*j],W[i,6*j+1],W[i,6*j+2]])
        v_solrb[i,j] = np.array([W[i,6*j+3],W[i,6*j+4],W[i,6*j+5]])
    rcom[i,0] = np.sum(m*r_solrb[i,:,0])/np.sum(m)
    rcom[i,1] = np.sum(m*r_solrb[i,:,1])/np.sum(m)
    rcom[i,2] = np.sum(m*r_solrb[i,:,2])/np.sum(m)
    vcom[i,0] = np.sum(m*v_solrb[i,:,0])/np.sum(m)
    vcom[i,1] = np.sum(m*v_solrb[i,:,1])/np.sum(m)
    vcom[i,2] = np.sum(m*v_solrb[i,:,2])/np.sum(m)
    for j in range(N):
        r_solrb[i,j] -= rcom[i]
        v_solrb[i,j] -= vcom[i]
    for j in range(N):
        for k in range(j+1,N):
            EPrb[i] += -m[j]*m[k]/sci.linalg.norm(r_solrb[i,j]-r_solrb[i,k])
        EKrb[i] += 1/2*m[j]*(v_solrb[i,j,0]**2+v_solrb[i,j,1]**2+v_solrb[i,j,2]**2)
E = EKrb+EPrb
error = np.zeros(len(E)-1)
logErr = np.zeros(len(E)-1)
for i in range(1,len(E)):
    error[i-1] = abs((E[i]-E[0])/E[0])
    if error[i-1] == 0:
        logErr[i-1] = -16
    else:
        logErr[i-1] = np.log10(error[i-1])
plt.figure(figsize=(6,6))
plt.plot(t[1:], logErr)
plt.xscale('log')
plt.xlabel('Waktu (thn)')
plt.xlim(t[1],t[-1])
plt.ylabel('Log Error')
plt.title('RK4 N = %d'%subres)
plt.grid(axis='both')
plt.show()

In [ ]:
#ModMid
subres = int(input('Masukkan subres = '))
mulai = time.time()
W = np.zeros((len(t),len(W0)))
W[0] = W0
h = t[1]-t[0]
for i in range(1,len(t)):
    W[i] = ModMid(lambda w, t: Gravity(w, t, m), W[i-1], t[i-1], h, subres)
selesai = time.time()
print('%f s'%(selesai-mulai))
EKm = np.zeros(len(t))
EPm = np.zeros(len(t))
r_solm = np.zeros((len(t),N, 3))
v_solm = np.zeros((len(t),N, 3))
rcom = np.zeros((len(t), 3))
vcom = np.zeros((len(t), 3))
for i in range(len(t)):
    for j in range(N):
        r_solm[i,j] = np.array([W[i,6*j],W[i,6*j+1],W[i,6*j+2]])
        v_solm[i,j] = np.array([W[i,6*j+3],W[i,6*j+4],W[i,6*j+5]])
    rcom[i,0] = np.sum(m*r_solm[i,:,0])/np.sum(m)
    rcom[i,1] = np.sum(m*r_solm[i,:,1])/np.sum(m)
    rcom[i,2] = np.sum(m*r_solm[i,:,2])/np.sum(m)
    vcom[i,0] = np.sum(m*v_solm[i,:,0])/np.sum(m)
    vcom[i,1] = np.sum(m*v_solm[i,:,1])/np.sum(m)
    vcom[i,2] = np.sum(m*v_solm[i,:,2])/np.sum(m)
    for j in range(N):
        r_solm[i,j] -= rcom[i]
        v_solm[i,j] -= vcom[i]
    for j in range(N):
        for k in range(j+1,N):
            EPm[i] += -m[j]*m[k]/sci.linalg.norm(r_solm[i,j]-r_solm[i,k])
        EKm[i] += 1/2*m[j]*(v_solm[i,j,0]**2+v_solm[i,j,1]**2+v_solm[i,j,2]**2)
E = EKm+EPm
error = np.zeros(len(E)-1)
logErr = np.zeros(len(E)-1)
for i in range(1,len(E)):
    error[i-1] = abs((E[i]-E[0])/E[0])
    if error[i-1] == 0:
        logErr[i-1] = -16
    else:
        logErr[i-1] = np.log10(error[i-1])
plt.figure(figsize=(6,6))
plt.plot(t[1:], logErr)
plt.xscale('log')
plt.xlabel('Waktu (thn)')
plt.xlim(t[1],t[-1])
plt.ylabel('Log Error')
plt.title('Modified Midpoint N = %d'%subres)
plt.grid(axis='both')
plt.show()

In [ ]:
#Bulisch-Stoer
subres = int(input('N = '))
mulai = time.time()
W = np.zeros((len(t),len(W0)))
W[0] = W0
h = t[1]-t[0]
W[1] = RE(lambda w, t: Gravity(w, t, m), lambda func, w, t, h: ModMid(func, w, t, h, subres), W[0], subres, h, oke=True)
for i in range(1,len(t)):
    W[i] = RE(lambda w, t: Gravity(w, t, m), lambda func, w, t, h: ModMid(func, w, t, h, subres), W[i-1], subres, h, oke=False)
selesai = time.time()
print('%f s'%(selesai-mulai))
#########RUTINITAS
EKmb = np.zeros(len(t))
EPmb = np.zeros(len(t))
r_solmb = np.zeros((len(t),N, 3))
v_solmb = np.zeros((len(t),N, 3))
rcom = np.zeros((len(t), 3))
vcom = np.zeros((len(t), 3))
for i in range(len(t)):
    for j in range(N):
        r_solmb[i,j] = np.array([W[i,6*j],W[i,6*j+1],W[i,6*j+2]])
        v_solmb[i,j] = np.array([W[i,6*j+3],W[i,6*j+4],W[i,6*j+5]])
    rcom[i,0] = np.sum(m*r_solmb[i,:,0])/np.sum(m)
    rcom[i,1] = np.sum(m*r_solmb[i,:,1])/np.sum(m)
    rcom[i,2] = np.sum(m*r_solmb[i,:,2])/np.sum(m)
    vcom[i,0] = np.sum(m*v_solmb[i,:,0])/np.sum(m)
    vcom[i,1] = np.sum(m*v_solmb[i,:,1])/np.sum(m)
    vcom[i,2] = np.sum(m*v_solmb[i,:,2])/np.sum(m)
    for j in range(N):
        r_solmb[i,j] -= rcom[i]
        v_solmb[i,j] -= vcom[i]
    for j in range(N):
        for k in range(j+1,N):
            EPmb[i] += -m[j]*m[k]/sci.linalg.norm(r_solmb[i,j]-r_solmb[i,k])
        EKmb[i] += 1/2*m[j]*(v_solmb[i,j,0]**2+v_solmb[i,j,1]**2+v_solmb[i,j,2]**2)
E = EKmb+EPmb
error = np.zeros(len(E)-1)
logErr = np.zeros(len(E)-1)
for i in range(1,len(E)):
    error[i-1] = abs((E[i]-E[0])/E[0])
    if error[i-1] == 0:
        logErr[i-1] = -16
    else:
        logErr[i-1] = np.log10(error[i-1])
plt.figure(figsize=(6,6))
plt.plot(t[1:], logErr)
plt.xscale('log')
plt.xlabel('Waktu (thn)')
plt.xlim(t[1],t[-1])
plt.ylabel('Log Error')
plt.title('Modified Midpoint N = %d'%subres)
plt.grid(axis='both')
plt.show()

In [ ]:
#KHUSUS 2 Benda
EK = np.zeros(len(t))
EP = np.zeros(len(t))
for i in range(len(t)):
    EK[i] = 0.5*m*V[i]**2
    EP[i] = -m1*m2/R[i]
E = EK+EP
error = np.zeros(len(E)-1)
logErr = np.zeros(len(E)-1)
for i in range(1,len(E)):
    error[i-1] = abs((E[i]-E[0])/E[0])
    if error[i-1] == 0:
        logErr[i-1] = -16
    else:
        logErr[i-1] = np.log10(error[i-1])
plt.figure(figsize=(6,6))
plt.plot(t[1:], logErr)
plt.xscale('log')
plt.xlabel('Waktu (thn)')
plt.xlim(t[1],t[-1])
plt.ylabel('Log Error')
plt.title('EKSAK')
plt.grid(axis='both')
plt.show()

In [ ]:
%matplotlib notebook
r_sol = r_solmb
v_sol = v_solmb
# R = np.zeros(len(t))
# for i in range(len(t)):
#     R[i] = sci.linalg.norm(r_sol[i,1])
# for i in range(len(t)):
#     for j in range(N):
#         r_sol[i,j] = Proj(r_sol[i,j], r_sol[0,1]-r_sol[0,0], v_sol[0,1]-v_sol[0,0])
#         v_sol[i,j] = Proj(v_sol[i,j], r_sol[0,1]-r_sol[0,0], v_sol[0,1]-v_sol[0,0])     
# print(r_sol[0,1])
# plt.plot(t,R)
plt.figure(figsize=(6,6))
plt.axis('equal')
plt.xlabel("X")
plt.ylabel("Y")
for i in range(N):
    plt.plot(r_sol[:,i,0]-r_sol[:,0,0],r_sol[:,i,1]-r_sol[:,0,1])
    plt.scatter(r_sol[-1,i,0]-r_sol[-1,0,0],r_sol[-1,i,1]-r_sol[-1,0,1])
    plt.scatter(r_sol[0,i,0]-r_sol[0,0,0],r_sol[0,i,1]-r_sol[0,0,1], marker='x')
plt.show()
plt.figure(figsize=(6,6))
plt.axis('equal')
plt.xlabel("X")
plt.ylabel("Z")
for i in range(N):
    plt.plot(r_sol[:,i,0]-r_sol[:,0,0],r_sol[:,i,2]-r_sol[:,0,2])
    plt.scatter(r_sol[-1,i,0]-r_sol[-1,0,0],r_sol[-1,i,2]-r_sol[-1,0,2])
    plt.scatter(r_sol[0,i,0]-r_sol[0,0,0],r_sol[0,i,2]-r_sol[0,0,2], marker='x')
plt.show()
plt.figure(figsize=(6,6))
plt.axis('equal')
plt.xlabel("Y")
plt.ylabel("Z")
for i in range(N):
    plt.plot(r_sol[:,i,1]-r_sol[:,0,1],r_sol[:,i,2]-r_sol[:,0,2])
    plt.scatter(r_sol[-1,i,1]-r_sol[-1,0,1],r_sol[-1,i,2]-r_sol[-1,0,2])
    plt.scatter(r_sol[0,i,1]-r_sol[0,0,1],r_sol[0,i,2]-r_sol[0,0,2], marker='x')
plt.show()
Error = abs(E[-1]-E[0])/abs(E[0])
print('Error = ',Error)

In [ ]:
fig=plt.figure(figsize=(10,10))
ax=fig.add_subplot(111,projection="3d")
for i in range(N):
    if i != 10:
        ax.plot(r_sol[:,i,0],r_sol[:,i,1],r_sol[:,i,2])
        ax.scatter(r_sol[-1,i,0],r_sol[-1,i,1],r_sol[-1,i,2])
        ax.scatter(r_sol[0,i,0],r_sol[0,i,1],r_sol[0,i,2],marker='x')
# pos = np.zeros((len(t),3))
# R = np.zeros(len(t))
# for i in range(len(t)):
#     pos[i] = Proj(r_sol[i,2], r_sol[i,1]-r_sol[i,0],v_sol[i,1]-v_sol[i,0])
#     R[i] = sci.linalg.norm(r_sol[i,1]-r_sol[i,0])
# ax.plot(pos[:,0],pos[:,1],pos[:,2], c='b')
# ax.plot(R,R*0,R*0, c='k')
# ax.scatter(pos[-1,0],pos[-1,1],pos[-1,2], c='c')
# ax.scatter(R[-1],0,0, c='k')
# ax.scatter(0,0,0, c='r')

ax.set_xlabel("x-coordinate",fontsize=14)
ax.set_ylabel("y-coordinate",fontsize=14)
ax.set_zlabel("z-coordinate",fontsize=14)
ax.set_xlim(-50,50)
ax.set_ylim(-50,50)
ax.set_zlim(-50,50)
# ax.set_axis('equal')
ax.set_title("Orbit Pluto dan Neptunus relatif terhadap pusat massa\n %d tahun"%rentang, fontsize=14)

In [ ]:
r_sol = rs
v_sol = vs
t = TIME
fig=plt.figure(figsize=(10,10))
ax=fig.add_subplot(111,projection="3d")
# for i in range(N):
#     if i != 10:
#         ax.plot(r_sol[:,i,0],r_sol[:,i,1],r_sol[:,i,2])
#         ax.scatter(r_sol[-1,i,0],r_sol[-1,i,1],r_sol[-1,i,2])
Rp = np.zeros((len(t),3))
Vp = np.zeros((len(t),3))
Rn = np.zeros((len(t),3))
Vn = np.zeros((len(t),3))
for i in range(len(t)):
    Rp[i] = Proj(r_sol[i,2], r_sol[i,1]-r_sol[i,0],v_sol[i,1]-v_sol[i,0])
    Vp[i] = Proj(v_sol[i,2], r_sol[i,1]-r_sol[i,0],v_sol[i,1]-v_sol[i,0])
    Rn[i] = Proj(r_sol[i,1], r_sol[i,1]-r_sol[i,0],v_sol[i,1]-v_sol[i,0])
    Vn[i] = Proj(v_sol[i,1], r_sol[i,1]-r_sol[i,0],v_sol[i,1]-v_sol[i,0])
ax.plot(Rp[:,0],Rp[:,1],Rp[:,2], c='b')
ax.plot(Rn[:,0],Rn[:,1],Rn[:,2], c='k')
ax.scatter(Rp[-1,0],Rp[-1,1],Rp[-1,2], c='c')
ax.scatter(Rn[-1,0],Rn[-1,1],Rn[-1,2], c='k')
ax.scatter(0,0,0, c='r')

ax.set_xlabel("x-coordinate",fontsize=14)
ax.set_ylabel("y-coordinate",fontsize=14)
ax.set_zlabel("z-coordinate",fontsize=14)
ax.set_xlim(-60,60)
ax.set_ylim(-60,60)
ax.set_zlim(-60,60)
# ax.set_axis('equal')
ax.set_title("Orbit Pluto relatif terhadap kerangka berputar Neptunus-Matahari\n %d tahun"%(250000), fontsize=14)

In [ ]:
fig=plt.figure(figsize=(10,10))
ax=fig.add_subplot(111,projection="3d")

ax.plot(Vp[:,0],Vp[:,1],Vp[:,2], c='b')
ax.plot(Vn[:,0],Vn[:,1],Vn[:,2], c='k')
ax.scatter(Vp[-1,0],Vp[-1,1],Vp[-1,2], c='c')
ax.scatter(Vn[-1,0],Vn[-1,1],Vn[-1,2], c='k')
ax.scatter(0,0,0, c='r')

ax.set_xlabel("vx-coordinate",fontsize=14)
ax.set_ylabel("vy-coordinate",fontsize=14)
ax.set_zlabel("vz-coordinate",fontsize=14)
ax.set_xlim(-0.2,0.2)
ax.set_ylim(-0.2,0.2)
ax.set_zlim(-0.2,0.2)
# ax.set_axis('equal')
ax.set_title("Vektor Kecepatan Pluto relatif terhadap kerangka berputar Neptunus-Matahari\n %d tahun"%(250000), fontsize=14)

In [ ]:
tulis = open('Pluto250000.txt','w')
t = np.arange(0,250000,2.5)
for i in range(len(t)):
    tulis.write(str(t[i])+' ')
    for j in range(N):
        tulis.write(str(r_sol[i,j,0])+' '+str(r_sol[i,j,1])+' '+str(r_sol[i,j,2])+' '+str(v_sol[i,j,0])+' '+str(v_sol[i,j,1])+' '+str(v_sol[i,j,2])+' ')
    tulis.write('\n')
tulis.close()

In [ ]:
Data = 'Pluto250000.txt'
Data0 = np.loadtxt(Data, max_rows=1)
time = np.loadtxt(Data, usecols=0)
N = (len(Data0)-1)//6
DW = np.zeros((len(time),len(Data0)-1))
for j in range(1,len(Data0)):
    DW[:,j-1] = np.loadtxt(Data, usecols=j)
rs = np.zeros((len(time),N, 3))
vs = np.zeros((len(time),N, 3))
for i in range(len(time)):
    for j in range(N):
        rs[i,j] = np.array([DW[i,6*j],DW[i,6*j+1],DW[i,6*j+2]])
        vs[i,j] = np.array([DW[i,6*j+3],DW[i,6*j+4],DW[i,6*j+5]])
        rs[i,j] -= rs[i,0]
        vs[i,j] -= vs[i,0]

In [ ]:
%matplotlib notebook
plt.figure(figsize=(6,6))
plt.axis('equal')
plt.xlabel("X")
plt.ylabel("Y")
plt.grid()
for i in range(N):
    plt.plot(rs[:,i,0],rs[:,i,1])
    plt.scatter(rs[-1,i,0],rs[-1,i,1])
    plt.scatter(rs[-1,i,0],rs[-1,i,1], marker='x')
plt.show()

plt.figure(figsize=(6,6))
plt.axis('equal')
plt.xlabel("X")
plt.ylabel("Z")
for i in range(N):
    plt.plot(rs[:,i,0],rs[:,i,2])
    plt.scatter(rs[-1,i,0],rs[-1,i,2])
    plt.scatter(rs[-1,i,0],rs[-1,i,2], marker='x')
plt.show()
plt.figure(figsize=(6,6))
plt.axis('equal')
plt.xlabel("Y")
plt.ylabel("Z")
for i in range(N):
    plt.plot(rs[:,i,1],rs[:,i,2])
    plt.scatter(rs[-1,i,1],rs[-1,i,2])
    plt.scatter(rs[-1,i,1],rs[-1,i,2], marker='x')
plt.show()

In [ ]:
TIME=time
Rp = np.zeros((len(TIME),3))
Vp = np.zeros((len(TIME),3))
Rn = np.zeros((len(TIME),3))
Vn = np.zeros((len(TIME),3))
for i in range(len(TIME)):
    Rp[i] = Proj(rs[i,2], rs[i,1],vs[i,1])
    Vp[i] = Proj(vs[i,2], rs[i,1],vs[i,1])
    Rn[i] = Proj(rs[i,1], rs[i,1],vs[i,1])
    Vn[i] = Proj(vs[i,1], rs[i,1],vs[i,1])

In [ ]:
rs[0,1]

In [ ]:
hp = np.cross(Rp,Vp)
hn = np.cross(Rn,Vn)
Hp = np.zeros(len(TIME))
Hn = np.zeros(len(TIME))
RP = np.zeros(len(TIME))
RN = np.zeros(len(TIME))
VP = np.zeros(len(TIME))
VN = np.zeros(len(TIME))
for i in range(len(TIME)):
    Hp[i] = sci.linalg.norm(hp[i])
    Hn[i] = sci.linalg.norm(hn[i])
    RP[i] = sci.linalg.norm(Rp[i])
    RN[i] = sci.linalg.norm(Rn[i])
    VP[i] = sci.linalg.norm(Vp[i])
    VN[i] = sci.linalg.norm(Vn[i])
ap = 1/(2/RP-VP**2)
ep = np.sqrt(1-Hp**2/ap)
an = 1/(2/RN-VN**2)
en = np.sqrt(1-Hn**2/an)

In [ ]:
mulai = time.time()
inc = np.zeros(len(TIME))
omega = np.zeros(len(TIME))
Omega = np.zeros(len(TIME))
zn = np.zeros((len(TIME),3))
zp = np.zeros((len(TIME),3))
om = np.zeros((len(TIME),3))
exn = np.zeros((len(TIME),3))
exp = np.zeros((len(TIME),3))
rp = np.zeros((len(TIME),3))
rn = np.zeros((len(TIME),3))
tp = np.zeros((len(TIME),3))
tn = np.zeros((len(TIME),3))
Tp = np.zeros(len(TIME))
Tn = np.zeros(len(TIME))
for j in range(len(TIME)):
    zn[j] = hn[j]/sci.linalg.norm(hn[j])
    zp[j] = hp[j]/sci.linalg.norm(hp[j])
    inc[j] = np.arccos(np.dot(hn[j],hp[j])/(Hn[j]*Hp[j]))
    om[j] = np.cross(zn[j],zp[j])/np.sin(inc[j])
    rp[j] = rs[j,2]/sci.linalg.norm(rs[j,2])
    rn[j] = rs[j,1]/sci.linalg.norm(rs[j,1])
    tp[j] = np.cross(zp[j],rp[j])
    tn[j] = np.cross(zn[j],rn[j])
    Tp[j] = THE(rs[j,2],vs[j,2],ap[j], ep[j])
    Tn[j] = THE(rs[j,1],vs[j,1],an[j], en[j])
    exn[j] = rn[j]*np.cos(Tn[j])-tn[j]*np.sin(Tn[j])
    exp[j] = rp[j]*np.cos(Tp[j])-tp[j]*np.sin(Tp[j]) 
    Omega[j] = Angle(exn[j],exp[j],hn[j],False)
    omega[j] = Angle(om[j], exp[j], hp[j],False)
selesai = time.time()
print(selesai-mulai)

In [ ]:
# sudut = np.linspace(0,2*np.pi,100)
# a = 1
# e = 0.2
# r = a*(1-e**2)/(1+e*np.cos(sudut))
# v = np.sqrt(2/r-1/a)
# alpha = np.arcsin(np.sqrt(a*(1-e**2))/r/v)
# vx = v*np.cos(alpha)
# vy = v*np.sin(alpha)
# rx = r*np.cos(sudut)
# ry = r*np.sin(sudut)
# ux = np.array([1,0,0])
# uy = np.array([0,1,0])
# R = np.zeros((len(sudut),3))
# V = np.zeros((len(sudut),3))
# for i in range(len(sudut)):
#     R[i] = rx[i]*ux+ry[i]*uy
#     V[i] = vx[i]*ux+vy[i]*uy

In [ ]:
# THE(R[0],V[0],a,e)
# RR = a*(1-e**2)/(1+e)
# (1-a*(1-e**2)/RR)/e

In [ ]:
# def THE(r,v,a,e):
#     R = sci.linalg.norm(r)
#     cos = (a*(1-e**2)/R-1)/e
#     if np.dot(v,r)>0:
#         return np.arccos(cos)
#     else:
#         return 2*np.pi-np.arccos(cos)
# z = np.array([0,0,1])
# ru = np.zeros((len(sudut),3))
# tu = np.zeros((len(sudut),3))
# ex = np.zeros((len(sudut),3))
# Tu = np.zeros(len(sudut))
# for j in range(len(sudut)):
#     ru[j] = R[j]/sci.linalg.norm(R[j])
#     tu[j] = np.cross(z,ru[j])
#     Tu[j] = THE(R[j],V[j],a, e)
#     ex[j] = ru[j]*np.cos(Tu[j])-tu[j]*np.sin(Tu[j])
plt.figure()
# plt.plot(sudut,Tu,'o')
plt.plot(TIME[:],omega[:])
# plt.plot(tu[:10,0],tu[:10,1],'o')
# plt.plot(ex[:10,0],ex[:10,1],'o')
# plt.axis('equal')
plt.grid()
plt.show()  
# plt.figure()
# plt.plot(sudut[:10],Tu[:10],'o')
# # plt.plot(ru[:10,0],ru[:10,1],'o')
# # plt.plot(tu[:10,0],tu[:10,1],'o')
# # plt.plot(ex[:10,0],ex[:10,1],'o')
# plt.axis('equal')
# plt.grid()
# plt.show()  

In [ ]:
Dp = np.zeros(len(TIME))
z = np.cross(exp[0],exp[1])
Dt = TIME[1]-TIME[0]
for i in range(1,len(TIME)):
    Dp[i] = Angle(exp[i-1],exp[i],z, True)/Dt
Dp[0] = Dp[1]

In [ ]:
Da = np.zeros(len(TIME))
z = np.cross(om[0],om[1])
Dt = TIME[1]-TIME[0]
for i in range(1,len(TIME)):
    Da[i] = Angle(om[i-1],om[i],z, True)/Dt
Da[0] = Da[1]

In [ ]:
L =100
zero = np.zeros(len(TIME))
dist = -(abs(zero-rs[:,2,2])-1)
from scipy.signal import find_peaks
potong, _ = find_peaks(dist,height=0)
pasli = np.zeros(len(potong))
indeks = 0
for i in potong:
    xtime = TIME[i-2:i+2]
    cek = np.linspace(xtime[0],xtime[-1],100)
    yval = rs[i-2:i+2,2,2]
    poly = np.polyfit(xtime,yval,1)
    pasli[indeks] = -poly[1]/poly[0]
    indeks += 1
plt.figure()
plt.plot(TIME[:],rs[:,2,2])
plt.plot(pasli,pasli*0,'x')
plt.grid()
plt.xlim(TIME[0],TIME[100])
plt.show()
print(pasli)

In [ ]:
xval = np.zeros(len(potong))
yval = np.zeros(len(potong))
indeks = 0
for i in potong:
    xtime = TIME[i-3:i+3]
    xv = rs[i-3:i+3,2,0]
    yv = rs[i-3:i+3,2,1]
    polyx = np.polyfit(xtime,xv,2)
    polyy = np.polyfit(xtime,yv,2)
    plt.plot(cek,np.poly1d(polyx)(cek))
    xval[indeks] = np.poly1d(polyx)(pasli[indeks])
    yval[indeks] = np.poly1d(polyy)(pasli[indeks])
    indeks += 1

In [ ]:
plt.figure()
plt.plot(TIME[:],rs[:,2,0])
plt.plot(pasli,xval,'x')
plt.grid()
plt.xlim(TIME[0],TIME[1000])
plt.show()
plt.figure()
plt.plot(TIME[:],rs[:,2,1])
plt.plot(pasli,yval,'x')
plt.grid()
plt.xlim(TIME[0],TIME[1000])
plt.show()

In [ ]:
Vec = np.array([xval,yval,xval*0]).transpose()
Da = np.zeros(len(potong)-1)
z = np.cross(Vec[0],Vec[1])
Dt = np.diff(pasli)
for i in range(len(potong)-1):
    Da[i] = Angle(Vec[i],Vec[i+1],z, True)/Dt[i]

In [ ]:
Tnaik = (pasli[0:-1]+pasli[1:])/2
print(Tnaik)
plt.figure()
plt.title('Pergeseran Titik Nodal Naik \norbit Pluto terhadap orbit Neptunus')
plt.plot(Tnaik,Da/np.diff(pasli)*180/np.pi,'o')
# plt.plot(TIME[:L],garis[:L],c='r')
plt.xlabel('Waktu (thn)')
plt.ylabel('deg')
plt.grid()
plt.xlim(Tnaik[0],Tnaik[100])
plt.show()

In [ ]:
da = np.zeros(len(Tnaik))
for i in range(len(da)):
    da[i] = np.trapz(Da[:i]/np.diff(pasli)[:i],Tnaik[:i])

In [ ]:
plt.figure()
plt.title('Pergeseran Titik Nodal Naik \norbit Pluto terhadap orbit Neptunus')
plt.plot(Tnaik[:],da[:]*180/np.pi)
# plt.plot(TIME[:L],garis[:L],c='r')
plt.xlabel('Waktu (thn)')
plt.ylabel('deg')
plt.grid()
plt.xlim(Tnaik[0],Tnaik[-1])
plt.show()

In [ ]:
plt.xlabel()

In [ ]:
L = 100000
# garis = TIME*(-1.927925716864314e-05)/TIME[-1]
plt.figure()
plt.title('Laju Pergeseran Perihelion Pluto')
plt.plot(TIME[:L],Dp[:L]*180/np.pi)
# plt.plot(TIME[:L],garis[:L],c='r')
plt.xlabel('Waktu (thn)')
plt.ylabel('deg/thn')
plt.grid()
plt.xlim(TIME[0],TIME[L-1])
plt.show()
print(np.sum(Dp*Dt))

In [ ]:
dp = np.zeros(len(TIME))
for i in range(len(dp)):
    dp[i] = np.trapz(Dp[:i],TIME[:i])
plt.figure()
plt.title('Pergeseran Perihelion Pluto')
plt.plot(TIME[:L],dp[:L])
# plt.plot(TIME[:L],garis[:L],c='r')
plt.xlabel('Waktu (thn)')
plt.ylabel('deg')
plt.grid()
plt.xlim(TIME[0],TIME[L-1])
plt.show()

In [ ]:
fig=plt.figure(figsize=(10,10))
ax=fig.add_subplot(111,projection="3d")
ax.plot(exp[:100,0],exp[:100,1],exp[:100,2], c='b')
ax.plot(exn[:100,0],exn[:100,1],exn[:100,2], c='c')
# ax.plot(hn[:,0],hn[:,1],hn[:,2], c='c', ls='--')
# ax.scatter(exn[0,0],exn[0,1],exn[0,2], c='c')
# ax.plot([0,om[0,0]],[0,om[0,1]],[0,om[0,2]], c='b')
# ax.scatter(om[0,0],om[0,1],om[0,2], c='c')
# ax.set_xlim(-1,1)
# ax.set_ylim(-1,1)
# ax.set_zlim(-1,1)
print(hn[:10])

In [ ]:
slib = np.zeros(len(TIME))
xdir = np.array([1.,0.,0.])
zdir = np.array([0.,0.,1.])
for i in range(len(TIME)):
    slib[i] = Angle(xdir,Rp[i],zdir, True)
132.15286169402149
46.90692446194058

In [ ]:
LIB1 = 132.15286169402149
LIB2 = 46.90692446194058
LIB3 = 58.3106
LIB1-LIB3

In [ ]:
L =1000
plt.figure()
plt.plot(TIME[:L],slib[:L]*180/np.pi)
plt.xlabel('Waktu (thn)')
plt.ylabel('Sudut (deg)')
plt.title('Sudut Neptunus-Pluto dari Matahari')
plt.xlim(TIME[0],TIME[L-1])
plt.show()
plt.ylim(58.3105, 58.3107)
# plt.xlim(10000,13500)
plt.axhline(58.3106, c='k')

In [ ]:
dt = TIME[1]-TIME[0]
dlib = np.diff(slib*100)/dt
t2 = (TIME[:-1]+TIME[1:])/2

In [ ]:
print(slib)

In [ ]:
plt.figure()
plt.plot(t2[:200],dlib[:200])
plt.plot(TIME[:200],slib[:200]*3)
plt.ylim(-10,10)
plt.show()
plt.axhline(0,c='k')

In [ ]:
L = len(TIME)//1
plt.figure(figsize=(10,5))
plt.title('Neptunus')
plt.plot(TIME[:L],an[:L])
plt.xlabel('Waktu (thn)')
plt.ylabel('a (sa)')
plt.xlim(TIME[0],TIME[L-1])
plt.show()
plt.figure(figsize=(10,5))
plt.title('Neptunus')
plt.plot(TIME[:L],en[:L])
plt.xlabel('Waktu (thn)')
plt.ylabel('e ')
plt.xlim(TIME[0],TIME[L-1])
plt.show()

plt.figure(figsize=(10,5))
plt.title('Pluto')
plt.plot(TIME[:L],ap[:L])
plt.xlabel('Waktu (thn)')
plt.ylabel('a (sa)')
plt.xlim(TIME[0],TIME[L-1])
plt.show()
plt.figure(figsize=(10,5))
plt.title('Pluto')
plt.plot(TIME[:L],ep[:L])
plt.xlabel('Waktu (thn)')
plt.ylabel('e ')
plt.xlim(TIME[0],TIME[L-1])
plt.show()
plt.figure(figsize=(10,5))
plt.title('Pluto')
plt.plot(TIME[:L],inc[:L]*180/np.pi)
plt.xlabel('Waktu (thn)')
plt.ylabel('inklinasi (deg) ')
plt.xlim(TIME[0],TIME[L-1])
plt.show()
plt.figure(figsize=(10,5))
plt.title('Pluto')
plt.plot(TIME[:L],Dp[:L]*180/np.pi)
plt.xlabel('Waktu (thn)')
plt.ylabel('Pergeseran Perihelion (deg) ')
plt.xlim(TIME[0],TIME[L-1])
plt.show()

In [ ]:
# R = np.zeros(len(time))
# for i in range(len(time)):
#     R[i] = sci.linalg.norm(Rp[i]-Rn[i])    


In [ ]:
L = len(time)//500
# plt.figure(figsize=(10,5))
# # plt.title('Pluto')
# plt.plot(time[:L],R[:L])
# plt.xlabel('Waktu (thn)')
# plt.ylabel('Jarak Pluto-Neptune (au) ')
# plt.xlim(time[0],time[L-1])
# plt.show()

plt.figure(figsize=(10,5))
# plt.title('Pluto')
plt.plot(time[:L],rps[:L])
plt.xlabel('Waktu (thn)')
plt.ylabel('Jarak Pluto-Matahari (au) ')
plt.xlim(time[0],time[L-1])
plt.show()

plt.figure(figsize=(10,5))
plt.title('Pluto terhadap Neptunus')
plt.plot(time[:L],inc[:L])
plt.xlabel('Waktu (thn)')
plt.ylabel('Inklinasi (deg) ')
plt.xlim(time[0],time[L-1])
plt.show()

In [ ]:
r1 = -m2/M*r
r2 = m1/M*r
v1 = -m2/M*v
v2 = m1/M*v
plt.figure(figsize=(5,5))
plt.axis('equal')
plt.grid()
plt.plot(r1[:,0],r1[:,1], label='Eksak', c='b')
plt.plot(r1[-1,0],r1[-1,1], 'x', c='b')
plt.plot(r_solE[:,0,0],r_solE[:,0,1], label='Euler', ls='--', c='k')
plt.plot(r_solE[-1,0,0],r_solE[-1,0,1], 'o', c='k')
plt.plot(r_solr[:,0,0],r_solr[:,0,1], label='RK4', ls='-.', c='r')
plt.plot(r_solr[-1,0,0],r_solr[-1,0,1], 'o', c='r')
plt.plot(r_solo[:,0,0],r_solo[:,0,1], label='ODEINT', ls='-.', c='g')
plt.plot(r_solo[-1,0,0],r_solo[-1,0,1], 'o', c='g')
plt.plot(r_solm[:,0,0],r_solm[:,0,1], label='ModMid', ls=':', c='y')
plt.plot(r_solm[-1,0,0],r_solm[-1,0,1], 'o', c='y')

plt.legend(loc='best')
plt.show()

plt.figure(figsize=(5,5))
# plt.axis('equal')
plt.grid()
plt.plot(r1[:,0],r1[:,1], label='Eksak', c='b')
plt.plot(r1[-1,0],r1[-1,1], 'x', c='b', ms=20)
plt.plot(r_solE[:,0,0],r_solE[:,0,1], label='Euler', ls='--', c='k')
plt.plot(r_solE[-1,0,0],r_solE[-1,0,1], 'o', c='k')
plt.plot(r_solr[:,0,0],r_solr[:,0,1], label='RK4', ls='-.', c='r')
plt.plot(r_solr[-1,0,0],r_solr[-1,0,1], 'o', c='r')
plt.plot(r_solo[:,0,0],r_solo[:,0,1], label='ODEINT', ls=':', c='g')
plt.plot(r_solo[-1,0,0],r_solo[-1,0,1], 'o', c='g')
plt.plot(r_solm[:,0,0],r_solm[:,0,1], label='ModMid', ls=':', c='y')
plt.plot(r_solm[-1,0,0],r_solm[-1,0,1], 'o', c='y')
plt.legend(loc='best')
plt.xlim(-0.525,-0.475)
plt.ylim(-0.05,0.05)
plt.show()

# plt.figure(figsize=(5,5))
# plt.axis('equal')
# plt.grid()
# plt.plot(v1[:,0],v1[:,1])
# plt.plot(v1[-1,0],v1[-1,1], 'o')
# plt.plot(v2[:,0],v2[:,1])
# plt.plot(v2[-1,0],v2[-1,1], 'o')
# plt.show()

In [ ]:
r1 = -m2/M*r
r2 = m1/M*r
v1 = -m2/M*v
v2 = m1/M*v
plt.figure(figsize=(5,5))
plt.axis('equal')
plt.grid()
plt.plot(r1[:,0],r1[:,1], label='Eksak', c='b')
plt.plot(r1[-1,0],r1[-1,1], 'x', c='b')
plt.plot(r_solEb[:,0,0],r_solEb[:,0,1], label='Euler+BS', ls='--', c='k')
plt.plot(r_solEb[-1,0,0],r_solEb[-1,0,1], 'o', c='k')
plt.plot(r_solrb[:,0,0],r_solrb[:,0,1], label='RK4+BS', ls='-.', c='r')
plt.plot(r_solrb[-1,0,0],r_solrb[-1,0,1], 'o', c='r')
plt.plot(r_solo[:,0,0],r_solo[:,0,1], label='ODEINT', ls='-.', c='g')
plt.plot(r_solo[-1,0,0],r_solo[-1,0,1], 'o', c='g')
plt.plot(r_solm[:,0,0],r_solm[:,0,1], label='MidMod+BS', ls=':', c='y')
plt.plot(r_solm[-1,0,0],r_solm[-1,0,1], 'o', c='y')
plt.legend(loc='best')
plt.show()

plt.figure(figsize=(5,5))
# plt.axis('equal')
plt.grid()
plt.plot(r1[:,0],r1[:,1], label='Eksak', c='c')
plt.plot(r1[-1,0],r1[-1,1], 'x', c='b', ms=20)
plt.plot(r_solEb[:,0,0],r_solEb[:,0,1], label='Euler+BS', ls='--', c='k')
plt.plot(r_solEb[-1,0,0],r_solEb[-1,0,1], 'o', c='k')
plt.plot(r_solrb[:,0,0],r_solrb[:,0,1], label='RK4+BS', ls='-.', c='r')
plt.plot(r_solrb[-1,0,0],r_solrb[-1,0,1], 'o', c='r')
plt.plot(r_solo[:,0,0],r_solo[:,0,1], label='ODEINT', ls=':', c='g')
plt.plot(r_solo[-1,0,0],r_solo[-1,0,1], 'o', c='g')
plt.plot(r_solmb[:,0,0],r_solmb[:,0,1], label='MidMod+BS', ls=':', c='y')
plt.plot(r_solmb[-1,0,0],r_solmb[-1,0,1], 'o', c='y')
plt.legend(loc='best')
plt.xlim(-0.50001,-0.49999)
plt.ylim(-0.00001,0.00001)
plt.show()

In [ ]:
sci.linalg.norm(r_sol[0,1])